**Importar dataset**

Convertir archivo CSV a HDF5

In [1]:
#Importar librerías necesarias

import pandas as pd
import numpy as np
import h5py
import os
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

#Obtener directorio actual y a partir de él, encontrar la ubicación del dataset en formato csv

current_dir = os.getcwd()
csv_path = os.path.join(current_dir, 'dataset', 'Train.csv')

#num_lines es el número de entradas y num_features es la cantidad de atributos (exceptuando la categoría)
num_lines = 750
num_features = 7

class_dict = {'low': 0, 'medium': 1, 'high': 2}
chunksize = 10

with h5py.File('Train.h5', 'w') as h5f:
    
  dset1 = h5f.create_dataset(
    'features',
    shape=(num_lines, num_features),
    compression=None,
    dtype='float32'
  )
  dset2 = h5f.create_dataset(
    'labels',
    shape=(num_lines,),
    compression=None,
    dtype='int32')

  for i in range(0, num_lines, chunksize):  
    
    df = pd.read_csv(
      csv_path,  
      header=None,  
      nrows=chunksize, 
      skiprows=i 
    )   
    
    df[7] = df[7].map(class_dict)

    features = df.values[:, :7]
    labels = df.values[:, -1]
    
    dset1[i:i+10, :] = features
    dset2[i:i+10] = labels

Implementar una clase Dataset personalizada

In [2]:
class Hdf5Dataset(Dataset):
  def __init__(self, h5_path, transform=None):
  
    self.h5f = h5py.File(h5_path, 'r')
    self.num_entries = self.h5f['labels'].shape[0]
    self.transform = transform

  def __getitem__(self, index):
      
    features = self.h5f['features'][index]
    label = self.h5f['labels'][index]
    if self.transform is not None:
      features = self.transform(features)
    return features, label

  def __len__(self):
    return self.num_entries

Inicializar DataLoader

In [3]:
train_dataset = Hdf5Dataset(h5_path='Train.h5', transform=None)

train_loader = DataLoader(
  dataset=train_dataset,
  batch_size=10,
  shuffle=True,
) 

In [4]:
'''
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.manual_seed(0)

num_epochs = 5
for epoch in range(num_epochs):

  for batch_idx, (x, y) in enumerate(train_loader):
      
    print('Epoch:', epoch+1, end='')
    print(' | Batch index:', batch_idx, end='')
    print(' | Batch size:', y.size()[0])
    
    x = x.to(device)
    y = y.to(device)
'''


'device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")\ntorch.manual_seed(0)\n\nnum_epochs = 5\nfor epoch in range(num_epochs):\n\n  for batch_idx, (x, y) in enumerate(train_loader):\n      \n    print(\'Epoch:\', epoch+1, end=\'\')\n    print(\' | Batch index:\', batch_idx, end=\'\')\n    print(\' | Batch size:\', y.size()[0])\n    \n    x = x.to(device)\n    y = y.to(device)'

**Desarrollo Guía 2**

In [5]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

#import helper
import matplotlib.pyplot as plt

dataiter = iter(train_loader)
images, labels = next(dataiter)
print(type(images))
print(images.shape)
print(labels.shape)

<class 'torch.Tensor'>
torch.Size([10, 7])
torch.Size([10])


In [ ]:
def softmax(x):
    return torch.exp(x)/torch.sum(torch.exp(x), dim=1).view(-1, 1)

probabilities = softmax(out)

# Does it have the right shape? Should be (64, 10)
print(probabilities.shape)
# Does it sum to 1?
print(probabilities.sum(dim=1))